In [1]:
import networkx as nx
from networkx.algorithms import bipartite
import sqlite3 
import json

def create_graph(group_id,list_retweeters , name_db):
    sqliteConnection = sqlite3.connect(name_db)
    cursor = sqliteConnection.cursor()
    bipartite_graph = nx.Graph()
   
    bipartite_graph.add_nodes_from(list_retweeters, bipartite=0)
    #print (tuple(set_retweeter))
    
    tuple_set_retweeter=tuple(list_retweeters)
    #print(tuple_set_retweeter)
    
    cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
    retweeter_tweet_ids_records = cursor.fetchall()
    set_tweet=[]
    set_tweet=[row[1] for row in retweeter_tweet_ids_records]
    bipartite_graph.add_nodes_from(set_tweet, bipartite=1)
    
    
    set_edges=[]
    for row in retweeter_tweet_ids_records:
        set_edges.append((row[0], row[1])) 
    bipartite_graph.add_edges_from(set_edges)
    
    retweeter_nodes = {n for n, d in bipartite_graph.nodes(data=True) if d['bipartite']==0}
    retweeter_group_graph = bipartite.weighted_projected_graph(bipartite_graph, retweeter_nodes,ratio=False)
    
    edges_less_weight = [(u, v,d) for (u, v, d) in retweeter_group_graph.edges(data=True) if d['weight'] < 3 ]
    retweeter_group_graph.remove_edges_from(edges_less_weight)
    
    
    #data  = graph_features(retweeter_group_graph,len(user_id), label , sn)
    
    print("group_id:  ",group_id)
    print(retweeter_group_graph.degree(weight='weight'))
    #data  = graph_features(retweeter_group_graph,len(list_retweeters) , group_id)
    #bipartite_graph.clear()
    return 

with open('rtg_Annual_bonus.json','r') as reader:
        
    for line in reader:
        #skip empty line
        if line.strip():
            
            retweeter_group_read=json.loads(line)
            group_id=retweeter_group_read["group_id"]
            members_id=retweeter_group_read["members_id"]
            
            create_graph(group_id,members_id ,'Annual_bonus.db')
            print("\n")
            print("\n")
            print("\n")    
        else:
            continue
                



group_id:   d91230bdd7cf435198ae4697f460aed0
[(1128675519262863360, 128), (1130599371001802753, 156), (837781737392467969, 175), (903730553274474496, 211), (700661264993771521, 160), (742560077933096961, 68), (1149849191235104769, 166), (958460184715759616, 43), (921770524635746304, 96), (1041730412597399552, 37), (892500838257758210, 62), (1149706342199959553, 38), (879224371063160832, 49), (948888751689826304, 30), (520411556, 135), (4543946661, 81), (543368108, 42), (2226713924, 47), (1277964114, 36), (2936379350, 109), (717275479, 177), (1131444036596883456, 57), (792729754055180288, 28), (2448134390, 35), (404368763, 22)]






group_id:   2e15a6d536fc4aafa02aa546b491bc97
[(1133096694176993280, 30), (1053569255734681600, 36), (1059461546319994882, 58), (1149835367442538496, 45), (744167420022104068, 54), (480863237, 52), (1059492910364872704, 30), (395803143, 55), (1061170321712144384, 47), (1134878604310011904, 36), (3307238634, 41), (1033089200290230283, 53), (1217517163, 43), (

In [12]:
import networkx as nx
from networkx.algorithms import bipartite
import sqlite3 
import json
import pandas as pd

def create_graph(group_id,list_retweeters , name_db):
    sqliteConnection = sqlite3.connect(name_db)
    cursor = sqliteConnection.cursor()
    bipartite_graph = nx.Graph()
   
    bipartite_graph.add_nodes_from(list_retweeters, bipartite=0)
    #print (tuple(set_retweeter))
    
    tuple_set_retweeter=tuple(list_retweeters)
    #print(tuple_set_retweeter)
    
    cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
    retweeter_tweet_ids_records = cursor.fetchall()
    set_tweet=[]
    set_tweet=[row[1] for row in retweeter_tweet_ids_records]
    bipartite_graph.add_nodes_from(set_tweet, bipartite=1)
    
    
    set_edges=[]
    for row in retweeter_tweet_ids_records:
        set_edges.append((row[0], row[1])) 
    bipartite_graph.add_edges_from(set_edges)
    
    retweeter_nodes = {n for n, d in bipartite_graph.nodes(data=True) if d['bipartite']==0}
    retweeter_group_graph = bipartite.weighted_projected_graph(bipartite_graph, retweeter_nodes,ratio=False)
    
    edges_less_weight = [(u, v,d) for (u, v, d) in retweeter_group_graph.edges(data=True) if d['weight'] < 3 ]
    retweeter_group_graph.remove_edges_from(edges_less_weight)
    
    
    #data  = graph_features(retweeter_group_graph,len(user_id), label , sn)
    
    print("group_id:  ",group_id)
    print(retweeter_group_graph.degree(weight='weight'))
    save_csv(retweeter_group_graph ,4)
    #data  = graph_features(retweeter_group_graph,len(list_retweeters) , group_id)
    #bipartite_graph.clear()
    return

def save_csv(sub ,count_group):
    
    #sub = connected_component.subgraph(finall_retweeters_ids)
    graph_information = pd.DataFrame(sub.edges.data())
    graph_information2 = pd.DataFrame(columns = ["source", "target","type" ,"weight"]) 
    graph_information2["source"]=graph_information[0].apply(lambda x: x)
    graph_information2["target"]=graph_information[1].apply(lambda x: x)
    graph_information2["weight"]=graph_information[2].apply(lambda x: x['weight'])
    graph_information2["type"]="Undirected"
    #file_name='%s_%s.csv' % ('Network_structure_group', time.strftime('%Y%m%d-%H%M%S'))
    file_name='%s_%s.csv' % ('Network_structure_group', count_group)
    graph_information2.to_csv(file_name,index=False,mode='w')
    return 

with open('rtg_hashtag_abdulaziz.json','r') as reader:
        
    for line in reader:
        #skip empty line
        if line.strip():
            
            retweeter_group_read=json.loads(line)
            group_id=retweeter_group_read["group_id"]
            members_id=retweeter_group_read["members_id"]
            if group_id=="80dc6d1a9945407da13ed4eec09a1771":
                #create_graph(group_id,members_id ,'hashtag_abdulaziz.db')
                print("\n")
                print("\n")
                print("\n")    
        else:
            continue
                




group_id:   80dc6d1a9945407da13ed4eec09a1771
[(1340769828, 344), (468276773, 344), (1398528804, 344), (1311184842, 344), (1024916107, 344), (2514880079, 344), (2346063958, 344), (2697600570, 344), (605037339, 344)]








In [2]:
import networkx as nx
from networkx.algorithms import bipartite
import sqlite3 
import json

def create_graph(list_retweeters , name_db):
    sqliteConnection = sqlite3.connect(name_db)
    cursor = sqliteConnection.cursor()
    bipartite_graph = nx.Graph()
   
    bipartite_graph.add_nodes_from(list_retweeters, bipartite=0)
    #print (tuple(set_retweeter))
    
    tuple_set_retweeter=tuple(list_retweeters)
    #print(tuple_set_retweeter)
    
    cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
    retweeter_tweet_ids_records = cursor.fetchall()
    set_tweet=[]
    set_tweet=[row[1] for row in retweeter_tweet_ids_records]
    bipartite_graph.add_nodes_from(set_tweet, bipartite=1)
    
    
    set_edges=[]
    for row in retweeter_tweet_ids_records:
        set_edges.append((row[0], row[1])) 
    bipartite_graph.add_edges_from(set_edges)
    
    retweeter_nodes = {n for n, d in bipartite_graph.nodes(data=True) if d['bipartite']==0}
    retweeter_group_graph = bipartite.weighted_projected_graph(bipartite_graph, retweeter_nodes,ratio=False)
    
    edges_less_weight = [(u, v,d) for (u, v, d) in retweeter_group_graph.edges(data=True) if d['weight'] < 3 ]
    retweeter_group_graph.remove_edges_from(edges_less_weight)
    
    
    #data  = graph_features(retweeter_group_graph,len(user_id), label , sn)
    
    
    print(retweeter_group_graph.degree(weight='weight'))
    #data  = graph_features(retweeter_group_graph,len(list_retweeters) , group_id)
    #bipartite_graph.clear()
    return
list_retweeters=[311965957, 631632417, 561681351, 226210957, 279718423, 494259016, 584091435, 193169007, 585839043, 2799444228, 276925775, 2289797405, 634625672, 607716668, 1111619503]
create_graph(list_retweeters , 'Retweet_manipulation3.db')

[(631632417, 85), (585839043, 62), (2799444228, 49), (311965957, 85), (561681351, 85), (494259016, 77), (634625672, 52), (584091435, 73), (226210957, 68), (193169007, 73), (276925775, 51), (1111619503, 52), (279718423, 85), (607716668, 52), (2289797405, 51)]


In [3]:
import networkx as nx
from networkx.algorithms import bipartite
import sqlite3 
import json
import pandas as pd


#base graph is directed graph
base_graph = nx.DiGraph()

def save_csv(sub ):
    
    #sub = connected_component.subgraph(finall_retweeters_ids)
    graph_information = pd.DataFrame(sub.edges.data())
    graph_information2 = pd.DataFrame(columns = ["source", "target","type" ,"weight"]) 
    graph_information2["source"]=graph_information[0].apply(lambda x: x)
    graph_information2["target"]=graph_information[1].apply(lambda x: x)
    #graph_information2["weight"]=graph_information[2].apply(lambda x: x['weight'])
    graph_information2["type"]="directed"
    #file_name='%s_%s.csv' % ('Network_structure_group', time.strftime('%Y%m%d-%H%M%S'))
    file_name='%s.csv' % ('MRG3_network')
    graph_information2.to_csv(file_name,index=False,mode='w')
    return 

def tw_rt_graph(list_nodes_retweeter,name_db):
   
    

    
    tuple_set_retweeter=tuple(list_nodes_retweeter)
    
    with sqlite3.connect(name_db) as conn:
        cursor = conn.cursor()    
        q="SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,)
        #cursor.execute("SELECT retweeter_id, tweet_id FROM retweet WHERE retweeter_id IN %s" % (tuple_set_retweeter,))
        for record in cursor.execute(q):
            if record!=None:
                base_graph.add_edge(record[0],record[1], weight=1)
        
    
    
    return

with open('rtg_hashtag_alhilal.json','r') as reader:
        
    for line in reader:
        #skip empty line
        if line.strip():
            
            retweeter_group_read=json.loads(line)
            group_id=retweeter_group_read["group_id"]
            members_id=retweeter_group_read["members_id"]
            label=retweeter_group_read["label"]
           # if group_id=="80dc6d1a9945407da13ed4eec09a1771":
            if label ==True:
                tw_rt_graph(members_id ,'Retweet_manipulation3.db')
                  
        else:
            continue
save_csv(base_graph )            